# response-level v3.ipynb
## Author: Elliot Pallister

AIM: DEVELOPING A FULLY FORMULATED RESPONSE LEVEL FEATURE VECTOR FOR VISp IN RESPONSE TO im036_r FOR SESSION ID: 1044385384

In [1]:
# Collecting necessary imports

# External imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Internal imports
from pareto.data_io import get_cache, get_session, get_unit_channels, get_spike_times, get_stimulus_presentations, get_units_by_area, get_trials
from pareto.preprocessing import make_psth_cube, get_image_trials, arrange_image_onsets_to_trial, group_stims_by_frame_index
from pareto.stats import visual_selectivity_filter, subtract_baseline, zscore, roc_analysis
from pareto.plotting import plot_pop_mean, mean_variance_scatter

/opt/anaconda3/envs/allensdk/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Importing the cache from AllenSDK
cache = get_cache()

# Using session ID 1044385384
session_id = 1044385384
session = get_session(session_id)

units = get_unit_channels(session)
trials = get_trials(session)
stimulus_presentations = get_stimulus_presentations(session)
spike_times = get_spike_times(session)

/opt/anaconda3/envs/allensdk/lib/python3.11/site-packages/hdmf/spec/namespace.py:583: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
core - cached version: 2.6.0-alpha, loaded version: 2.7.0
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignored_namespaces)


Firstly, I want to filter my units based on:

1. Quality metrics (SNR, interspike interval violations and firing rate)
2. Area (starting with VISp)

In [3]:
quality_unit_filter = ((units['snr'] >= 1) & (units['isi_violations'] < 1) & (units['firing_rate'] > 0.1))
quality_units = units.loc[quality_unit_filter].copy()

area_of_interest = ['VISp']
area_units = get_units_by_area(quality_units, area_of_interest)

print(f'Number of filtered units in {area_of_interest}: {area_units.shape[0]}')


Number of filtered units in ['VISp']: 84


Next, I want to find the stimulus onsets for the image with id: im036_r.

I will use these onsets to:

1. Statistically test the unit responses for selectivity to the image, calculating effect sizes and p values, using a Wilcoxon paired rank test
2. Update the area units dataframe to contain effect sizes and p values in a column
3. Filter the area units by selectivity for the stimulus

In [4]:
stim_of_interest = 'im036_r'
stim_onsets = stimulus_presentations[stimulus_presentations['image_name'] == stim_of_interest]['start_time'].values

# Define the time before the image and the duration of the window during which spikes are counts
time_before_stim = 0.25
duration = 0.5

# Statistical testing
selectivity_mask, effects, p_values = visual_selectivity_filter(area_units, spike_times, stim_onsets, time_before_stim, duration)

area_units = area_units.copy()

# Assigning effect size and p_values
area_units.loc[:, 'p_values'] = p_values
area_units.loc[:, 'effect_size'] = effects

visual_area_units = area_units[selectivity_mask]

print(f'Number of units selective for {stim_of_interest} in {area_of_interest}: {visual_area_units.shape[0]}')

Number of units selective for im036_r in ['VISp']: 54


Next I want to produce a cube of dimensions U x O x T where U is the unit, O is the onset time and T is the time bin (50ms) within a set window around stimulus onset

In [ ]:
pre_onset_post_end = (0.25, 0, 0.25, 0.5)

cube, unit_ids, bins = make_psth_cube(visual_area_units, spike_times, stim_onsets, pre_onset_post_end, bin_size=0.01)
bins = bins[:-1]

print(isinstance(cube, np.ndarray))
print(cube.ndim)
print(cube.shape)

True
3
(54, 1064, 75)
[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [1 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 1 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 1 ... 0 0 0]
  [0 0 0 ... 1 1 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 1 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 1 ... 0 0 0]
  [0 0 0 ... 0 1 0]
  [1 0 0 ... 0 0 0]]]


Now that I have all of the individual data points, I will begin by baseline subtracting and z-scoring each of these traces and extract some information out by plotting a few important parameters. Here are the following questions I am looking to answer:

1. What is the distribution of mean firing rate taken across all stimulus presentations across all units?
2. What is the variance of firing rate and what is the distribution of this across the population?

In [ ]:
baseline_window = [-0.25, 0]
baseline_window = np.array(baseline_window)

bsub = subtract_baseline(cube, baseline_window, bins)

zs, mu, std = zscore(bsub)
mu = np.squeeze(mu)
std = np.squeeze(std)
var = std ** 2


evoked_window = np.array([0, 0.25])
evoked_indices = np.searchsorted(bins, evoked_window)
 

[ 0 25]
[0.06484962 0.06578947 0.07236842 0.06578947 0.1343985  0.1268797
 0.1287594  0.15601504 0.13815789 0.12406015 0.10902256 0.09116541
 0.08270677 0.08364662 0.1156015  0.09492481 0.10526316 0.10526316
 0.12969925 0.10244361 0.1156015  0.1137218  0.1137218  0.09210526
 0.09210526]


My next aim is to now build a feature vector for each unit and store this in a Pandas dataframe. Each vector will contain the following metrics:

1. Mean evoked response
2. Variance of evoked response
3. SNR
5. Mean AUC for trial chunks compared to first (ROC ANALYIS)
6. Mean AUC for consecutive trial chunks (ROC ANALYSIS)
7. Tuning based selectivity index
8. Population sparsity

In [41]:
def get_snr(
    responses,
    baseline_window, 
    evoked_window,
    bins
    ):

  bi = np.searchsorted(bins, baseline_window)
  ei = np.searchsorted(bins, evoked_window)

  baseline = responses[:, bi[0]:bi[1]].mean(axis=1)
  evoked = responses[:, ei[0]:ei[1]].mean(axis=1)

  bsub = evoked - baseline
  signal = bsub.mean()
  noise = np.std(bsub)

  snr = signal / noise

  return snr


snrs = []

for unit_responses in cube:
  snr = get_snr(unit_responses, baseline_window, evoked_window, bins)
  snrs.append(snr)

print(snrs)

def population_sparseness(
    responses,
    baseline_window,
    evoked_window,
    bins
    ):
  
  bi = np.searchsorted(bins, baseline_window)
  ei = np.searchsorted(bins, evoked_window)

  baseline = responses[:, :, bi[0]:bi[1]].mean(axis=1)
  evoked = responses[:, :, ei[0]:ei[1]].mean(axis=1)

  bsub = evoked - baseline

  mu_evoked = bsub.mean(axis=1)

  print(mu_evoked.shape)

  num = (1 - ((mu_evoked.mean() ** 2)/((mu_evoked ** 2).mean())))
  den = (1 - (1 / mu_evoked.shape[0]))

  sparsity = num / den

  return sparsity

sparsity = population_sparseness(cube, baseline_window, evoked_window, bins)

print(f"The population sparsity for {stim_of_interest} is {sparsity}")

[0.7134546830438268, 0.16715315054612778, 0.4020209164778138, 0.07256120498678784, 0.14342672818547325, 0.44664493474076616, 0.21954291147060423, 0.015101873905110919, 0.08408074438453143, 0.26168865216300263, -0.2879116606122853, 0.44723546932260777, 0.09801230327650937, 0.18507856625433655, -0.2646023634525721, 0.3606902431366869, 0.006824002729944793, 0.26205895680966507, 0.23018390257684826, 0.3975854153578544, 0.3943405431185601, -0.16259208451933063, 0.2809619987616095, 1.04319962899487, -0.10966952147332083, 0.25930606625367536, 0.374288335228282, 0.6906463064927258, 0.6956907457834272, 0.5229058058990597, 0.41083531185435934, 0.1835921647473505, 0.9224917802163763, -0.01875032441644379, 0.25013717422157017, 0.3485588035285266, 0.3549246017468689, -0.040355116277904765, 0.8392384114777222, 0.7568303874415098, -0.04042065017220026, 0.5295144358191571, -0.07646791870860466, 0.13570751369585904, 0.22128400223081443, 0.31422925849732875, 0.4364708577362672, 1.130966032663564, 1.0613

In [ ]:
def auc_roc(responses, chunk_size):

  n_chunks = int(round(responses.shape[1]/chunk_size))

  unit_chunks = []

  for unit in responses:

    chunks = []

    for i in range(n_chunks):
      chunk = unit[i*20:(i+1)*20]
      chunks.append(chunk)
  
    chunks = np.array(chunks)
    unit_chunks.append(chunks)

  unit_chunks = np.array(unit_chunks)

  unit_TPRs, unit_FPRs, unit_AUCs = [], [], []

  for unit in unit_chunks:

    TPRs, FPRs, AUCs = [], [], []

    for c in range(n_chunks-1):

      TPR, FPR = roc_analysis(unit[0], unit[c+1])
      TPR, FPR = np.array(TPR), np.array(FPR)
      TPRs.append(TPR)
      FPRs.append(FPR)
      order = np.argsort(FPR)
      auc = np.trapz(TPR[order], FPR[order]) 
      if auc < 0.5:
        auc = 1 - auc
      AUCs.append(auc)

    TPRs, FPRs, AUCs = np.array(TPRs), np.array(FPRs), np.array(AUCs)
    unit_TPRs.append(TPRs)
    unit_FPRs.append(FPRs)
    unit_AUCs.append(AUCs)

  unit_TPRs, unit_FPRs, unit_AUCs = np.array(unit_TPRs), np.array(unit_FPRs), np.array(unit_AUCs)

  return unit_TPRs, unit_FPRs, unit_AUCs

evoked_responses = bsub[:,:,evoked_indices[0]:evoked_indices[1]].mean(axis=2)

TPRs, FPRs, AUCs = auc_roc(evoked_responses, 20)

print(len(AUCs))
print(len(snrs))

means = evoked_responses.mean(axis=1)
variances = evoked_responses.var(axis=1)

print(means)
print(variances)

54
54
[ 0.04432331  0.0112782   0.03206767  0.00225564  0.00353383  0.03052632
  0.00759398  0.00142857  0.01451128  0.01721804 -0.01827068  0.02439849
  0.00394737  0.01003759 -0.0162782   0.02240601  0.00026316  0.01180451
  0.01304511  0.01958646  0.01793233 -0.00943609  0.01233083  0.08943609
 -0.00503759  0.02        0.01774436  0.05469925  0.03740601  0.03898496
  0.0181203   0.01278195  0.05578947 -0.00109023  0.015       0.02218045
  0.03890977 -0.0024812   0.05586466  0.03041353 -0.00195489  0.04699248
 -0.00421053  0.00394737  0.00522556  0.01672932  0.01721804  0.10924811
  0.09233083  0.02849624  0.03018797  0.01883459  0.00548872  0.03578947]
[0.0038595  0.0045525  0.00636264 0.00096634 0.00060706 0.00467115
 0.00119647 0.00894834 0.02978642 0.0043291  0.00402708 0.00297614
 0.00162201 0.00294135 0.00378464 0.00385887 0.00148715 0.00202907
 0.00321178 0.0024269  0.0020679  0.0033681  0.00192615 0.00735006
 0.00210996 0.00594887 0.00224754 0.00627265 0.00289102 0.00555837
 

In [ ]:
mean_AUCs = AUCs.mean(axis=1)

unit_features = pd.DataFrame(columns=['mean_evoked', 'evoked_variance', 'snr', 'evoked_AUC_compared_to_first'])

unit_features['mean_evoked'] = means
unit_features['evoked_variance'] = variances
unit_features['snr'] = snrs
unit_features['evoked_AUC_compared_to_first'] = mean_AUCs

print(unit_features.head())


   mean_evoked  evoked_variance       snr  evoked_AUC_compared_to_first
0     0.044323         0.003860  0.713455                      0.588534
1     0.011278         0.004553  0.167153                      0.696274
2     0.032068         0.006363  0.402021                      0.658654
3     0.002256         0.000966  0.072561                      0.599183
4     0.003534         0.000607  0.143427                      0.540240
0.10924811
